Hello. This is the 8th place solution. I'm sorry for posting the solution this *late*.

My path through this competition had 2 main stages. One important thing to have in mind is that in this competition I used my laptop, which has a limited hardware (Intel i5-5200U, 16 GB) so I always tried to minimize the number of columns, choose the right types and store partial results in pickle files to avoid my computer hang.

I'll briefly explain the first stage:

At first I promised myself to work with this paradigm: let's explore de data, let's formulate an hypothesis on how the data works and let's have it in mind the whole time because this could help me to detect bugs and making better decision on what things to try and what not.

My first thought was that the different version variables (software versions, database version, etc) could be an important key to predict Malware correctly, because they might hide some temporary information.

In [ ]:
import pandas as pd
import seaborn as sns
import gc
import numpy as np

dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'str',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'str',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [ ]:
train_df = pd.read_csv('../input/microsoft-malware-prediction/train.csv', dtype=dtypes)

In [ ]:
def category_and_version(category_col, version_col, df):
    aux = df[[category_col, 'HasDetections']].groupby(category_col).agg(['sum', 'count'])
    aux['infected_rate'] = aux['HasDetections']['sum'] / aux['HasDetections']['count']
    aux['sum'] = aux['HasDetections']['sum']
    aux['count'] = aux['HasDetections']['count']
    aux = aux[['infected_rate', 'sum', 'count']].reset_index().sort_values('infected_rate', ascending=False)        
    
    category_vals = aux[category_col]
    
    print(aux)
    
    gc.collect()
    
    aux1 = df[[version_col, category_col, 'HasDetections']]
    
    for i_cat in category_vals:
        aux = aux1[aux1[category_col] == i_cat]
        ordered_levels = np.sort(np.unique(aux[version_col]))

        aux = aux[[version_col, 'HasDetections']].groupby(version_col).agg(['sum','count'])
        aux['infected_rate'] = aux['HasDetections']['sum'] / aux['HasDetections']['count']
        aux['sum'] = aux['HasDetections']['sum']
        aux['count'] = aux['HasDetections']['count']
        aux = aux[['infected_rate', 'sum', 'count']].reset_index()

        sns.utils.plt.figure(figsize=(15, 10))
        ax = sns.pointplot(x=version_col, y='infected_rate', palette="YlGnBu_d", #hue = 'count', 
                           data=aux, join=False, order=ordered_levels)
        ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
#         sns.utils.plt.legend(loc='upper left',bbox_to_anchor=(1, 1))
        ax.set_title(category_col + ' = ' + str(i_cat) + ', ' + str(version_col) + ' ordered')
        sns.utils.plt.show()

In [ ]:
category_and_version('ProductName', 'EngineVersion', train_df)

In [ ]:
category_and_version('ProductName', 'AppVersion', train_df)

In [ ]:
category_and_version('Platform', 'OsSuite', train_df)

Here I only plotted some variables, but in some cases (specially in those with most observations) I can see the infection rate is slightly higher for the oldest versions and for the newest versions too. Plus, in these cases infection rates through versions are close.

So given this information, I decided that one way to encode categorical variables could be creating a count variable, a infection rate variable and rolling window features (window size = 5) like mean and standard deviation infection rate. 

I did so, and the first stage I mentioned before had this way of encoding categorical variables was the core of my approach. I used other numerical variables too, plus I added variables to have in mind the order of the versions. I fitted a Gradient Boosting Machine and... 

I got bad results, ~0.64 as a public score.

Some things I tried too and didn't work:
- I created other variables like discretize variables using k-means or a decision tree
- I created new variables like display diagonal, display aspect rate, squared or rectangular screen shape, binary variables that say if two variables are equal or not (Census_OSArchitecture vs Processor, OsBuild vs Census_OSBuildRevision and SkuEdition vs Census_OSSkuName. 
- I even scrapped this site 'https://www.microsoft.com/en-us/wdsi/definitions/antimalware-definition-release-notes' to get more info about versions (severities and threat types)
- I got information from Google Trends for the topic 'malware' from 2012 to 2017.

Still, none of this worked.

When I reached this point I said to myself: "Okay, none of this worked so let's turn the 'competition mode' on". And this is the second stage.

I began by taking a look at other people kernels and I got this ideas:
- I wasn't using many categorical variables, plus my way of encoding variables wasn't the best 
- Many people could be overfitting
- The training and test distributions could be different

Among many kernels, this was the most useful for me: https://www.kaggle.com/fabiendaniel/detecting-malwares-with-lgbm 

Thanks to this kernel I discovered that I could use categorical columns without creating dummy variables, making the categorical variables encoding process way more easy and maybe even keeping almost all the "raw information power". Another thing I was doing differently compared to this kernel is that this kernel is that I was only using the training dataset when calculating count variables and Fabien uses both the traning and the test datasets (this is what made me thought that train and test distributions could be different and maybe using both datasets for this task could be an advantage).

Given all this, my main goal was not to overfit. How did I do this?
- I focused on not having big differences between the ROC AUC values from the different dataset splits 
- Manually choosing which variables should be used in the model (and which categorical features should be encoded as counts, this was key too)
- I felt comfortable with a 0.66 ROC AUC as public score. I distrusted higher public score values because when I got those values I had big differences between my train and validation splits, and sometimes this happened at the cost of adding more variables because adding only one variable caused very little improvement.

Here's the code that creates the desired variables for train and test datasets:

In [ ]:
# data_dtypes.py
DTYPES = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category', 
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8', 
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8', 
        'DefaultBrowsersIdentifier':                            'float16', 
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16', 
        'HasTpm':                                               'int8', 
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16', 
        'GeoNameIdentifier':                                    'float16', 
        'LocaleEnglishNameIdentifier':                          'int8', 
        'Platform':                                             'category', 
        'Processor':                                            'category', 
        'OsVer':                                                'category', 
        'OsBuild':                                              'int16', 
        'OsSuite':                                              'int16', 
        'OsPlatformSubRelease':                                 'category', 
        'OsBuildLab':                                           'category', 
        'SkuEdition':                                           'category', 
        'IsProtected':                                          'float16', 
        'AutoSampleOptIn':                                      'int8', 
        'PuaMode':                                              'category', 
        'SMode':                                                'float16', 
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16', 
        'UacLuaenable':                                         'float32', 
        'Census_MDC2FormFactor':                                'category', 
        'Census_DeviceFamily':                                  'category', 
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16', 
        'Census_ProcessorManufacturerIdentifier':               'float16', 
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category', 
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category', 
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8', 
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category', 
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16', 
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16', 
        'Census_PowerPlatformRoleName':                         'category', 
        'Census_InternalBatteryType':                           'category', 
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category', 
        'Census_OSArchitecture':                                'category', 
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16', 
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category', 
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16', 
        'Census_OSUILocaleIdentifier':                          'int16', 
        'Census_OSWUAutoUpdateOptionsName':                     'category', 
        'Census_IsPortableOperatingSystem':                     'int8', 
        'Census_GenuineStateName':                              'category', 
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16', 
        'Census_IsFlightsDisabled':                             'float16', 
        'Census_FlightRing':                                    'category', 
        'Census_ThresholdOptIn':                                'float16', 
        'Census_FirmwareManufacturerIdentifier':                'float16', 
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8', 
        'Census_IsWIMBootEnabled':                              'float16', 
        'Census_IsVirtualDevice':                               'float16', 
        'Census_IsTouchEnabled':                                'int8', 
        'Census_IsPenCapable':                                  'int8', 
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16', 
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [ ]:
# feature_engineering.py
import numpy as np 
import pandas as pd 
from data_dtypes import DTYPES
from tqdm import tqdm

TRAIN_OUTPUT_PATH = './train_kaggle_feature_engineered.pickle'
TEST_OUTPUT_PATH = './test_kaggle_feature_engineered.pickle'

# https://gist.github.com/eherrerosj/fdac2b3761a10332da163b1c90367890
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def frequency_encoding(variable):
    t = pd.concat([train[variable], test[variable]]).value_counts().reset_index()
    t = t.reset_index()
    t.loc[t[variable] == 1, 'level_0'] = np.nan
    t.set_index('index', inplace=True)
    max_label = t['level_0'].max() + 1
    t.fillna(max_label, inplace=True)
    return t.to_dict()['level_0']

cols_to_use = ['MachineIdentifier', 'HasDetections', 
               'EngineVersion', 'AppVersion', 'RtpStateBitfield', 'AVProductsInstalled', 'IeVerIdentifier', 
               'SmartScreen', 'Census_OEMNameIdentifier', 'Census_ProcessorModelIdentifier', 
               'Census_PrimaryDiskTotalCapacity', 'Census_SystemVolumeTotalCapacity', 'Census_TotalPhysicalRAM', 
               'Census_InternalPrimaryDiagonalDisplaySizeInInches', 'Census_InternalBatteryNumberOfCharges', 
               'Census_OSBranch', 'Census_OSBuildRevision', 'Census_OSSkuName', 'Census_OSInstallTypeName', 
               'Census_ActivationChannel', 'Wdft_IsGamer', 'Wdft_RegionIdentifier']

categorical_features = ['SmartScreen', 'Census_OSInstallTypeName', 'Census_ActivationChannel',
                        'Census_OSBranch', 'Census_OSSkuName']

train = pd.read_csv('./train.csv', dtype = DTYPES, usecols=cols_to_use)
cols_to_use.remove('HasDetections')
test = pd.read_csv('./test.csv', dtype = DTYPES, usecols=cols_to_use)

frequency_encoded_variables = [
    'Census_ProcessorModelIdentifier',
    'Census_OEMNameIdentifier',
    'AppVersion', 
    'EngineVersion'
]

for variable in tqdm(frequency_encoded_variables):
    freq_enc_dict = frequency_encoding(variable)
    train[variable] = train[variable].map(lambda x: freq_enc_dict.get(x, np.nan))
    test[variable] = test[variable].map(lambda x: freq_enc_dict.get(x, np.nan))
    train[variable] = train[variable].astype('float16')
    test[variable] = test[variable].astype('float16')   

indexer = {}
for col in tqdm(categorical_features):
    print(col)
    if col == 'MachineIdentifier': continue
    _, indexer[col] = pd.factorize(train[col])
    
for col in tqdm(categorical_features):
    if col == 'MachineIdentifier': continue
    train[col] = indexer[col].get_indexer(train[col])
    test[col] = indexer[col].get_indexer(test[col])

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

train.to_pickle(TRAIN_OUTPUT_PATH)
test.to_pickle(TEST_OUTPUT_PATH)


Here's the code that trains the model and predicts the probabilities for the submission dataset:

In [ ]:
# train_and_predict.py
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
import seaborn as sns
from joblib import load, dump
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import roc_auc_score

TRAIN_OUTPUT_PATH = './train_kaggle_feature_engineered.pickle'
TEST_OUTPUT_PATH = './test_kaggle_feature_engineered.pickle'

# Reading previously feature engineered train dataset
train = pd.read_pickle(TRAIN_OUTPUT_PATH)

target = train['HasDetections']
del train['HasDetections']

# Creating splits
shuffle_split_tr = ShuffleSplit(n_splits=1, test_size=0.3, random_state=22)
shuffle_split_tr.get_n_splits(train, target)
train_idx, val_idx = next(shuffle_split_tr.split(train, target)) 

shuffle_split_val_test = ShuffleSplit(n_splits=1, test_size=0.5, random_state=22)
shuffle_split_val_test.get_n_splits(val_idx, target[val_idx])
val_idx, tst_idx = next(shuffle_split_val_test.split(val_idx, target[val_idx])) 

features = [c for c in train.columns if c not in ['MachineIdentifier']]

# Training stage
categorical_features = ['SmartScreen', 'Census_OSInstallTypeName', 'Census_ActivationChannel',
                        'Census_OSBranch', 'Census_OSSkuName', 'Wdft_RegionIdentifier', 
                        'Census_OSBuildRevision', 'IeVerIdentifier']

train_lgb = lgb.Dataset(train.iloc[train_idx][features],
                        label=target.iloc[train_idx],
                        categorical_feature = categorical_features
                       )

val_lgb = lgb.Dataset(train.iloc[val_idx][features],
                      label=target.iloc[val_idx],
                      categorical_feature = categorical_features
                     )

params = {'num_leaves': 60,
          'min_data_in_leaf': 60, 
          'objective':'binary',
          'max_depth': 8,
          'learning_rate': 0.1,
          'boosting': 'gbdt',
          'feature_fraction': 0.8,
          'bagging_freq': 1,
          'bagging_fraction': 0.8 ,
          'bagging_seed': 11,
          'metric': 'auc',
          'lambda_l1': 0.50,
          'random_state': 133,
          'verbosity': -1}

num_round = 3000
model = lgb.train(params,
                  train_lgb,
                  num_round,
                  valid_sets = [train_lgb, val_lgb],
                  verbose_eval = 100,
                  early_stopping_rounds = 200)                                                                                                                                                                          

dump(model, './malware_model_lgbm.pkl')

del train_lgb
del val_lgb

# Simple assessment
print('Training dataset:')
print( roc_auc_score(target[train_idx], model.predict(train.iloc[train_idx][features])) )

print('Validation dataset:')
print( roc_auc_score(target[val_idx], model.predict(train.iloc[val_idx][features])) )

print('Test dataset:')
print( roc_auc_score(target[tst_idx], model.predict(train.iloc[tst_idx][features])) )

del train

# Predicting probabilities for the submission dataset
test = pd.read_pickle(TEST_OUTPUT_PATH)
probs = model.predict(test[features], num_iteration=model.best_iteration)

sub_df = pd.DataFrame({'MachineIdentifier': test['MachineIdentifier'].values})
sub_df['HasDetections'] = probs

sub_df.to_csv('./malware_probs_submission.csv', index=False)

Variable importance:

In [ ]:
model = load('../input/malware_detection_challenge_8th_place_solutions/malware_model_lgbm.pkl')

importance_df = pd.DataFrame()
importance_df["feature"] = model.feature_name()
importance_df["importance"] = model.feature_importance(importance_type='gain')
pd.set_option('display.max_rows', 500)
importance_df.sort_values(ascending=False, by='importance')

Census_ProcessorModelIdentifier, Census_OEMNameIdentifier, AppVersion and EngineVersion were encoded as counts by category.

I didn't spend many time tweaking the model hyperparameters, to be honest: in this challenge I wanted to test some things I learnt at work: the data is the core of everything and working toe to tow with it can have a bigger impact in results than focusing on tweaking hyperparameters.

Maybe if I used k-fold Cross Validation I could have gotten better results.

Constructive criticism and feedback is well appreciated. Thank you!